# `QSDsan` Workshop Interactive Module <a class="anchor" id="top"></a>

- **Prepared by:**
    
    - [Yalin Li](https://qsdsan.readthedocs.io/en/latest/authors/Yalin_Li.html)
    
You can run the entire workbook and follow the prompts at the end.

In [1]:
#!!! Delete when testing with the released `qsdsan`
import os, sys
pwd = sys.path[0]
for abbr in ('tmo', 'bst', 'qs'):
    sys.path.append(os.path.abspath(os.path.join(pwd, f'../{abbr}')))

In [25]:
from ipywidgets import widgets as w
from country_specific import val_dct_cached, get_val_df, get_results, plot

# Placeholders for outputs
choice_out = w.Output()
data_out = w.Output()
result_out = w.Output()
data_result_outs = (data_out, result_out)
all_outs = (choice_out, data_out, result_out)

########## Setup displays ##########
# Error prompts
error_lbl = w.Label('')

# Let user choose whether to input data
style = {'description_width': 'initial'}
choice_btn = w.RadioButtons(
    options=['Yes', 'No'], 
    index=0,
    description='Would you like to use database values?',
    style=style, disabled=False)

# In the case of using database values
country_lbl = w.Label('Please enter a country name:')
country_txt = w.Text('')
country_btn = w.Button(
    description=' Retrieve Data',
    disabled=False,
    button_style='success', # 'success', 'info', 'warning', 'danger' or ''
    tooltip='Run database values for the parameters',
    icon='play' # (FontAwesome names without the `fa-` prefix)
)
param_lbl = w.Label()

# In the case of user-inputting values
input_lbl = w.Label('Please enter values for the following parameters:')
caloric_intake = w.Combobox(
    value='2130',
    placeholder='baseline 2130 [kcal/d]',
    description='Caloric intake:',
    style=style, disabled=False)

vegetable_protein = w.Combobox(
    value='40.29',
    placeholder='baseline 40.29 [g/d]',
    description='Vegetable protein intake:',
    style=style, disabled=False)

animal_protein = w.Combobox(
    value='12.39',
    placeholder='baseline 12.39 [g/d]',
    description='Animal protein intake:',
    style=style, disabled=False)

N_price = w.Combobox(
    value='1.507',
    placeholder='baseline 1.507 [USD/kg N]',
    description='N fertilizer price:',
    style=style, disabled=False)

P_price = w.Combobox(
    value='3.983',
    placeholder='baseline 3.983 [USD/kg P]',
    description='P fertilizer price:',
    style=style, disabled=False)

K_price = w.Combobox(
    value='1.333',
    placeholder='baseline 1.333 [USD/kg K]',
    description='K fertilizer price:',
    style=style, disabled=False)

food_waste_ratio = w.Combobox(
    value='0.02',
    placeholder='baseline 0.02 [fraction]',
    description='Food waste ratio:',
    style=style, disabled=False)

price_level_ratio = w.Combobox(
    value='1',
    placeholder='baseline 1',
    description='Price level ratio:',
    style=style, disabled=False)

income_tax = w.Combobox(
    value='0.3',
    placeholder='baseline 0.3 [fraction]',
    description='Income tax:',
    style=style, disabled=False)

all_inputs = (
    caloric_intake,
    vegetable_protein,
    animal_protein,
    N_price,
    P_price,
    K_price,
    food_waste_ratio,
    price_level_ratio,
    income_tax,
)

customize_btn = w.Button(
    description=' Confirm Data',
    disabled=False,
    button_style='success', # 'success', 'info', 'warning', 'danger' or ''
    tooltip='Confirm the input data for simulation',
    icon='play' # (FontAwesome names without the `fa-` prefix)
)

# Economic weight slider
econ_wt_lbl = w.Label('Please use the slider to set the economic weight in MCDA')
econ_wt_slider = w.FloatSlider(
    value=0.5,
    min=0.,
    max=1.,
    description='Weight: ',
    disabled=False,
    continuous_update=True,
    orientation='horizontal',
    readout=True,
    readout_format='.2f',
)
result_lbl = w.Label('')

# Simulate button
simulate_btn = w.Button(
    description=' Simulate',
    disabled=False,
    button_style='success', # 'success', 'info', 'warning', 'danger' or ''
    tooltip='Simulate and show results',
    icon='play' # (FontAwesome names without the `fa-` prefix)
)

# Clear outputs
clear_btn = w.Button(
    description=' Clear Outputs',
    disabled=False,
    button_style='warning', # 'success', 'info', 'warning', 'danger' or ''
    tooltip='Clear all outputs',
    icon='eraser' # (FontAwesome names without the `fa-` prefix)
)

########## Control displays ##########
def clear_results(btn=None):
    result_out.clear_output()

def clear_data_results(btn=None):
    for out in data_result_outs:
        out.clear_output()

def clear_all_outs(btn=None):
    for out in all_outs:
        out.clear_output()

def display_database_data(btn):
    global VALID
    data_out.clear_output()
    error_lbl.disabled = True
    with data_out:
        country = country_txt.value         
        global val_dct, val_df
        val_df = get_val_df(country)
        if isinstance(val_df, str): # no info for the country
            error_lbl.value = f'No data for country "{country}", please retry with another country name.'
            error_lbl.disabled = False
            display(error_lbl)
            VALID = False
        else:
            error_lbl.disabled = True
            param_lbl.value = f'Parameter values for {country}:'
            VALID = True
        update_simulate_btn(VALID)
    if VALID:
        with data_out:
            display(param_lbl)
            display(val_df)
            display(econ_wt_lbl)
            display(econ_wt_slider)
            display(simulate_btn)

            
def display_customized_inputs():
    choice_out.clear_output()
    with choice_out:
        val_dct = val_dct_cached.get('customized')
        val_dct = {} if val_dct is None else val_dct
        for i in all_inputs:
            display(i)
            val_dct[i.description[:-1]] = float(i.value)

def display_customized_data(btn=None):
    global VALID
    data_out.clear_output()
    with data_out:
        global country
        country_txt.value = country = 'customized'
        global val_dct, val_df
        val_df = get_val_df(country)
        param_lbl.value = f'Customized parameter values:'
        update_simulate()
        display(param_lbl)
        display(val_df)
        display(econ_wt_lbl)
        display(econ_wt_slider)
        display(simulate_btn)

def simulate(btn):
    update_simulate()
    if btn.button_style != 'success': return
    
    global results_dct
    country = country_txt.value
    weight = econ_wt_slider.value
    results_dct = get_results(country)
    with data_out:
        print(f'Results for {country}, economic weight is {weight}:')
    ax = plot(results_dct, econ_wt_slider.value)
    result_out.clear_output()
    with result_out:
        # display(ax.figure) the figure will be automatically displayed
        display(clear_btn)
            
def update_simulate():
#    global VALID
    VALID = True
    if choice == 'No':
        global country, val_dct
        country = 'customized'
        val_dct = {}
        for i in all_inputs:
            try:
                val_dct[i.description[:-1]] = float(i.value)
                val_dct_cached[country] = val_dct
            except:
                with data_out:
                    print(f'The value of input {i.description[:-1]} is "{i.value}", '
                         'not valid.')
                    VALID = False
                break
    update_simulate_btn(VALID)

def update_simulate_btn(VALID):
    if VALID:
        simulate_btn.button_style = 'success'
        simulate_btn.description = ' Simulate'
        simulate_btn.icon = 'play'
    else:
        simulate_btn.button_style = 'danger'
        simulate_btn.description = ' Invalid Inputs'
        simulate_btn.icon = 'stop'
        
def update_choice(btn=None):
    clear_all_outs()
    global choice
    choice = choice_btn.value
    if choice == 'Yes':
        with choice_out:
            display(country_lbl)
            display(country_txt)
            display(country_btn)
        country_btn.on_click(display_database_data)
    else:
        with choice_out:
            display(input_lbl)
            display_customized_inputs()
            display(customize_btn)
        customize_btn.on_click(display_customized_data)
            
choice_btn.observe(update_choice)
simulate_btn.on_click(simulate)
clear_btn.on_click(clear_data_results)

########## Compiled run function with the prompts and outs ##########
def run():
    update_choice() # initialize `choice_out` display
    display(choice_btn)
    display(choice_out)
    display(data_out)
    display(result_out)

In [26]:
run()

RadioButtons(description='Would you like to use database values?', options=('Yes', 'No'), style=DescriptionSty…

Output()

Output()

Output()